## My practice exercise of Joel Grus' example
from http://joelgrus.com/2016/05/23/fizz-buzz-in-tensorflow/

My modifications:
1. added one more hidden layer
2. the binary encode array enlarge to 15 digits: 2^15 = 32,768

In [1]:
import numpy as np
import tensorflow as tf

In [2]:
'''Encode the numbers as a binary array with given digits
Note: x & 1 is for deciding if the x is 1 or 0 (odd or even)
    >> is bitwise move from left to right
'''
def binary_encode(i, num_digits):
    return np.array([i >> d & 1 for d in range(num_digits)])[::-1]

In [3]:
''' representations of Y'''
def fizz_buzz_encode(i):
    if   i % 3 == 0: return np.array([0, 0, 0, 1])
    elif i % 5 == 0: return np.array([0, 0, 1, 0])
    elif i %15 == 0: return np.array([0, 1, 0, 0])
    else:            return np.array([1, 0, 0, 0])

In [4]:
NUM_DIGITS = 15

''' creating the training dataset, using list comprehension to create np.array
if the range is 2^10 = 1,024
if the range is 2^15 = 32,768
if the range is 2^20 = 1,048,576
'''
trX = np.array([binary_encode(i, NUM_DIGITS) for i in range(1, 2**NUM_DIGITS)])
trY = np.array([fizz_buzz_encode(i)          for i in range(1, 2**NUM_DIGITS)])

In [5]:
NUM_HIDDEN = 100

X = tf.placeholder("float", [None, NUM_DIGITS])
Y = tf.placeholder("float", [None, 4])

In [6]:
def init_weights(shape):
    return tf.Variable(tf.random_normal(shape, stddev=0.01))

w_in2hd  = init_weights([NUM_DIGITS, NUM_HIDDEN])
w_hd2hd  = init_weights([NUM_HIDDEN, NUM_HIDDEN])
w_hd2out = init_weights([NUM_HIDDEN, 4])

In [7]:
def model(X, w_in2hd, w_hd2hd, w_hd2out):
    h1 = tf.nn.relu(tf.matmul(X, w_in2hd))
    h2 = tf.nn.relu(tf.matmul(h1, w_hd2hd))
    output_layer = tf.matmul(h2, w_hd2out)
    return output_layer

In [8]:
py_x = model(X, w_in2hd, w_hd2hd, w_hd2out)

In [9]:
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=py_x, labels=Y))
train_op = tf.train.GradientDescentOptimizer(0.05).minimize(cost)

In [10]:
# The predict for us to see

predict_op = tf.argmax(py_x, 1)

In [11]:
def fizz_buzz(i, prediction):
    output_cat = [str(i), 'fizz', 'buzz', 'fizzbuzz']
    return output_cat[prediction]

In [12]:
BATCH_SIZE = 128
NUM_EPOCH = 1500

# create the graph in a session
with tf.Session() as sess:
    tf.initialize_all_variables().run()
    
    for epoch in range(NUM_EPOCH):
        # Shuffle the data
        p = np.random.permutation(range(len(trX)))
        trX, trY = trX[p], trY[p]
        
        # Train in batch size with 128 inputs
        for start in range(0, len(trX),BATCH_SIZE):
            end = start + BATCH_SIZE
            sess.run(train_op, feed_dict={X: trX[start:end], Y: trY[start:end]})
        
        # print current accuracy in each epoch
        if epoch % 100 == 0:
            print(epoch, np.mean(np.argmax(trY, axis=1) ==
                             sess.run(predict_op, feed_dict={X: trX, Y: trY})))
    
    # predict Fizz Buzz
    numbers = np.arange(1, 101)
    teX = np.transpose(binary_encode(numbers, NUM_DIGITS))
    teY = sess.run(predict_op, feed_dict={X: teX})
    output = np.vectorize(fizz_buzz)(numbers, teY)
    
    print(output)

Instructions for updating:
Use `tf.global_variables_initializer` instead.
0 0.533341471603
100 0.533341471603
200 0.651997436445
300 0.878383739738
400 0.99362163152
500 0.998596148564
600 0.999267555773
700 0.999694814905
800 0.999969481491
900 0.999969481491
1000 1.0
1100 1.0
1200 1.0
1300 1.0
1400 1.0
['1' '2' 'fizzbuzz' '4' 'buzz' 'fizzbuzz' '7' '8' 'fizzbuzz' 'buzz' '11'
 'fizzbuzz' '13' '14' 'fizzbuzz' '16' '17' 'fizzbuzz' '19' 'buzz'
 'fizzbuzz' '22' '23' 'fizzbuzz' 'buzz' '26' 'fizzbuzz' '28' '29'
 'fizzbuzz' '31' '32' 'fizzbuzz' '34' 'buzz' 'fizzbuzz' '37' '38'
 'fizzbuzz' 'buzz' '41' 'fizzbuzz' '43' '44' 'fizzbuzz' '46' '47'
 'fizzbuzz' '49' 'buzz' 'fizzbuzz' '52' '53' 'fizzbuzz' 'buzz' '56'
 'fizzbuzz' '58' '59' 'fizzbuzz' '61' '62' 'fizzbuzz' '64' 'buzz'
 'fizzbuzz' '67' '68' 'fizzbuzz' 'buzz' '71' 'fizzbuzz' '73' '74'
 'fizzbuzz' '76' '77' 'fizzbuzz' '79' 'buzz' 'fizzbuzz' '82' '83'
 'fizzbuzz' 'buzz' '86' 'fizzbuzz' '88' '89' 'fizzbuzz' '91' '92'
 'fizzbuzz' '94' 'buzz' '